# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import requests
import pandas as pd
import math

In [2]:
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [3]:
stocks=pd.read_csv("C:\\Users\\LENOVO\\Downloads\\algorithmic-trading-python-master\\sp_500_stocks.csv")
type(stocks)

pandas.core.frame.DataFrame

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [4]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [5]:
symbol="AAPL"
api_url=f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
data=requests.get(api_url).json()
type(data)
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': '/DALTA QA)NESESSLO  LTGG(CAREBNKM', 'calculationPrice': 'close', 'open': None, 'openTime': None, 'openSource': 'olfifaci', 'close': None, 'closeTime': None, 'closeSource': 'ifcaloif', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 124.21, 'latestSource': 'Close', 'latestTime': 'December 7, 2020', 'latestUpdate': 1615068387339, 'latestVolume': None, 'iexRealtimePrice': 128.74, 'iexRealtimeSize': 105, 'iexLastUpdated': 1655934292283, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 127.53, 'previousVolume': 81352813, 'change': 1.6, 'changePercent': 0.01254, 'volume': None, 'iexMarketPercent': 801.9075276859168, 'iexVolume': 1334512, 'avgTotalVolume': 103557816, 'iexBidPrice': 

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [6]:
price= data['latestPrice']
market_cap=data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [7]:
my_columns=['Ticker','Stock Price','Market Captlization','Number of Shares to Buy']
final_dataframe=pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Captlization,Number of Shares to Buy


In [8]:
final_dataframe.append(pd.Series([symbol,price,market_cap,'N/A'],index=my_columns,),ignore_index=True )

,Ticker,Stock Price,Market Captlization,Number of Shares to Buy
0,AAPL,124.21,2169839072309,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [9]:
final_dataframe=pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
    api_url=f"https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}"
    data=requests.get(api_url, proxies={"http": "http://111.233.225.166:1234"}).json()
    final_dataframe = final_dataframe.append(pd.Series([stock,data['latestPrice'],data['marketCap'],'N/A'],
                                                       index=my_columns),ignore_index=True)

In [10]:
final_dataframe

,Ticker,Stock Price,Market Captlization,Number of Shares to Buy
0,A,115.20,36350124746,N/A
1,AAL,17.58,10879457611,N/A
2,AAP,157.98,10840455481,N/A
3,AAPL,126.72,2165441059733,N/A
4,ABBV,110.39,190409993187,N/A
...,...,...,...,...
500,YUM,109.84,31930821373,N/A
501,ZBH,147.42,30453238319,N/A
502,ZBRA,389.04,20290551237,N/A
503,ZION,43.62,7382448465,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [11]:
def chunks(l, n):    
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 

In [12]:
symbol_groups=list(chunks(stocks['Ticker'],100))
symbol_strings=[]
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])
final_dataframe=pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url=f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    #print(batch_api_call_url)
    data=requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe=final_dataframe.append(pd.Series([symbol,data[symbol]["quote"]
                                                          ["latestPrice"],data[symbol]["quote"]["marketCap"],"N/A"],
                                                         index=my_columns),ignore_index=True)
final_dataframe

,Ticker,Stock Price,Market Captlization,Number of Shares to Buy
0,A,119,35585845588,N/A
1,AAL,17.27,10774750431,N/A
2,AAP,155.6,10885541744,N/A
3,AAPL,129.42,2168867546482,N/A
4,ABBV,108.07,195542047003,N/A
...,...,...,...,...
500,YUM,106.63,32738598396,N/A
501,ZBH,150.45,30612406738,N/A
502,ZBRA,388.19,20698318660,N/A
503,ZION,45.01,7065968880,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [16]:
portfolio_size=input('enter the value of your portfolio')

try:
    val=float(portfolio_size)
except ValueError:
    print('thats not a number! \nplease try again')
    portfolio_size=input('enter the value of your portfolio')
    

enter the value of your portfolio1000000


In [17]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])
final_dataframe

,Ticker,Stock Price,Market Captlization,Number of Shares to Buy,Number Of Shares to Buy
0,A,119,35585845588,N/A,16.0
1,AAL,17.27,10774750431,N/A,114.0
2,AAP,155.6,10885541744,N/A,12.0
3,AAPL,129.42,2168867546482,N/A,15.0
4,ABBV,108.07,195542047003,N/A,18.0
...,...,...,...,...,...
500,YUM,106.63,32738598396,N/A,18.0
501,ZBH,150.45,30612406738,N/A,13.0
502,ZBRA,388.19,20698318660,N/A,5.0
503,ZION,45.01,7065968880,N/A,43.0


In [18]:
final_dataframe.drop(['Number of Shares to Buy'], axis = 1) 

,Ticker,Stock Price,Market Captlization,Number Of Shares to Buy
0,A,119,35585845588,16.0
1,AAL,17.27,10774750431,114.0
2,AAP,155.6,10885541744,12.0
3,AAPL,129.42,2168867546482,15.0
4,ABBV,108.07,195542047003,18.0
...,...,...,...,...
500,YUM,106.63,32738598396,18.0
501,ZBH,150.45,30612406738,13.0
502,ZBRA,388.19,20698318660,5.0
503,ZION,45.01,7065968880,43.0


In [19]:
final_dataframe[['Ticker','Stock Price','Market Captlization','Number Of Shares to Buy']].to_csv("Equal_weight_S&P_500.csv",index=False)